# Compare fitted spectra with StarLight

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-11-27
- last update : 2023-11-27

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import pickle 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  16

## imports

### fitter jaxopt

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

In [ ]:
from dsps.cosmology import DEFAULT_COSMOLOGY, age_at_z

### parameters

In [ ]:
from fors2tostellarpopsynthesis.parameters import SSPParametersFit,paramslist_to_dict

### StarLight

In [ ]:
from fors2tostellarpopsynthesis.fors2starlightio import SLDataAcess, flux_norm

## Configuration

### fitted params

In [ ]:
input_path = "fitssp_results"
input_file = "fitssp_results.h5"

In [ ]:
fullname_input = os.path.join(input_path ,input_file) 

## Read StarLight

In [ ]:
sl = SLDataAcess()

## Read Dataframe 

In [ ]:
df = pd.read_hdf(fullname_input)

In [ ]:
df

In [ ]:
N = len(df)

In [ ]:
df

## Decode fitted parameters and plot spectra

In [ ]:
names_fitparams = list(df.columns[8:])

In [ ]:
all_x = []
all_y_nodust = []
all_y_dust = []
the_ymax= 1e-12

all_x_sl = []
all_y_sl = []

for idx in range(N):
    row = df.iloc[idx]
    z_obs = row["zobs"]
    specname = row["fors2name"]
    list_params = row[names_fitparams].values
    dict_params = paramslist_to_dict(list_params,names_fitparams)
    x,y_nodust,y_dust = ssp_spectrum_fromparam(dict_params,z_obs)

    norm_y_nodust = flux_norm(x,y_nodust)
    norm_y_dust = flux_norm(x,y_dust)

    y_nodust /=  norm_y_nodust
    y_dust /=  norm_y_dust
    
    dict_sl = sl.getspectrum_fromgroup(specname)
    x_sl,y_sl = dict_sl["wl"],dict_sl["fnu"]

    norm_sl = flux_norm(x_sl,y_sl)
    y_sl /= norm_sl

    
    all_x_sl.append(x_sl)
    all_y_sl.append(y_sl)

    ymax = y_nodust.max()
    the_ymax = max(the_ymax,ymax)
    
    all_x.append(x)
    all_y_nodust.append(y_nodust) 
    all_y_dust.append(y_dust) 

ylim_max = the_ymax*2.
ylim_min = ylim_max/1e7

In [ ]:
_, (ax1,ax2) = plt.subplots(2, 1,figsize=(10,8))


__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(N):
    ax1.plot(all_x[idx],all_y_nodust[idx],'-')

    #ax.plot(all_x_sl[idx],all_y_sl[idx])

__= ax1.set_xlim(100.,1e6)
__= ax1.set_ylim(ylim_min ,ylim_max )

ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax1.set_title("$f_\\nu(\lambda)$ fitted spectra")
ax1.grid()


__= ax2.set_yscale('log') 
__= ax2.set_xscale('log') 

for idx in range(N):
    ax2.plot(all_x_sl[idx],all_y_sl[idx],':')
__= ax2.set_xlim(100.,1e6)
__= ax2.set_ylim(ylim_min ,ylim_max )
ax2.grid()
ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax1.set_title("$f_\\nu(\lambda)$ Starlight spectra")


plt.tight_layout()

In [ ]:
_, ax = plt.subplots(1, 1)
__= ax.set_yscale('log') 
__= ax.set_xscale('log') 

for idx in range(N):
    ax.plot(all_x[idx],all_y_nodust[idx]/all_x[idx]**2)
__= ax.set_xlim(100.,1e5)
__= ax.set_ylim(1e-12 ,1e-6 )

ax.set_xlabel("$\lambda (\\AA)$")
ax.set_ylabel("$L_\\lambda(\lambda)$ (AB units - maggies")
ax.set_title("$f_\\lambda(\lambda)$ fitted spectra")
ax.grid()